<a href="https://colab.research.google.com/github/InoriNatsume/simple_colab_cataloganalyzer/blob/main/catalog3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 0. 구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')


# 1. 저장소로 부터 다운
!wget https://github.com/InoriNatsume/simple_colab_cataloganalyzer/archive/refs/heads/main.zip
!unzip main.zip
!mv simple_colab_cataloganalyzer-main hbr-asset-analyzer


# 2. 프로젝트 폴더로 이동하여 필요한 라이브러리 설치
# %cd는 Colab의 작업 디렉토리를 변경하는 명령어입니다.
%cd hbr-asset-analyzer
!pip install -r requirements.txt

print("\n\n✅ 프로젝트 설정 및 라이브러리 설치가 완료되었습니다.")

# 현재 작업 폴더를 확인하기 위해 셸 명령어를 단독으로 실행합니다.
print("현재 작업 폴더:")
!pwd



#3. 키 가져오기
from google.colab import userdata
import os

# Colab 보안 노트에서 키 값을 가져와 환경 변수로 설정
# 이 방식은 코드에 키가 직접 노출되지 않아 안전합니다.
os.environ['AES_KEY_HEX'] = userdata.get('AES_KEY_HEX')
os.environ['AES_IV_HEX'] = userdata.get('AES_IV_HEX')

print("✅ AES Key/IV를 Colab 보안 노트에서 성공적으로 불러왔습니다.")



# 4. Cloudflare 터널 설치 (이전과 동일)
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb > /dev/null 2>&1

# 8. Gradio 웹 애플리케이션을 백그라운드에서 먼저 실행
print("Gradio 서버를 백그라운드에서 시작합니다...")
get_ipython().system_raw('python frontend/app.py &')
import time
time.sleep(3) # Gradio 서버가 시작될 때까지 잠시 대기

# 5. Gradio 웹 서버를 위한 터널을 백그라운드에서 실행하고, 출력을 로그 파일에 저장
print("Cloudflare 터널을 시작합니다...")
get_ipython().system_raw('cloudflared tunnel --url http://127.0.0.1:7860 --no-autoupdate > /dev/null 2> cloudflared.log &')

# 6. 터널 URL이 생성될 때까지 충분히 대기 (10초로 늘림)
print("외부 접속 주소를 찾는 중입니다...")
time.sleep(10)

# 7. 로그 파일에서 생성된 외부 접속 URL을 찾아 화면에 출력
print("\n" + "="*50)
print("🎉 외부 접속 주소를 찾았습니다! 아래 주소로 접속하세요:")
!grep "trycloudflare.com" cloudflared.log
print("="*50 + "\n")

# Gradio 앱이 백그라운드에서 계속 실행 중이므로 이 셀은 완료됩니다.
# Gradio 프로세스를 중지하려면 런타임 -> 세션 관리에서 세션을 종료해야 합니다.
